# Download GEE Imagery

## Step-by-step tutorial

### Import libraries

In [33]:
import ee
import math
import geemap
import sys
import pandas as pd
from multiprocessing import Process, cpu_count 
from geetools import cloud_mask

In [34]:
Map = geemap.Map()
kazbegi = ee.FeatureCollection("users/andihanzl/kazbegi");
Map.centerObject(kazbegi, 10)
geometry =ee.Geometry.Polygon(
        [[[44.16644286751663, 42.600127655106185],
          [44.23510741829788, 42.5394457299407],
          [44.38067626595413, 42.48073042443687],
          [44.54547118782913, 42.456418303462],
          [44.72674560189163, 42.46452339297412],
          [44.87231444954788, 42.533374290609736],
          [44.90252685189163, 42.61023556954571],
          [44.80364989876663, 42.76367391712233],
          [44.56469726204788, 42.80398943064146],
          [44.16369628548538, 42.70315142463515]]]);
Map

Map(center=[42.6252601637589, 44.544784542321295], controls=(WidgetControl(options=['position'], widget=HBox(c…

In [35]:
snow = ee.ImageCollection('MODIS/006/MOD10A1') \
            .merge(ee.ImageCollection('MODIS/006/MYD10A1')) \
            .select(['NDSI_Snow_Cover'],['Snow']) \
            .filterBounds(kazbegi) \
            .filterDate('2000-02-26', '2000-12-31')

res = snow.first().projection()

snow = snow.map(lambda image: image.reproject(res).clip(kazbegi))

rad = ee.ImageCollection('ECMWF/ERA5_LAND/HOURLY') \
        .filterDate('2002-12-31', '2003-01-02') \
        .select(['surface_solar_radiation_downwards']) \
        .sort('system:time_start')

rad = rad.map(lambda image: image.resample('bilinear').reproject(res).clip(geometry).rename('RAD'))
rad = rad.sort('system:time_start')

rad = rad.filterMetadata('hour', 'equals', 10);

#Get the timestamp and convert it to a date.
hour = rad.first().get('hour')
print('Hour: '+str(hour.getInfo())) # ee.Date

count = rad.size()
print('Count: ', count.getInfo())

vis_params = {
      'min': -20,
      'max': 30,
      'palette': ['blue', 'red'],
      'bands': ['RAD']
}

#Map.addLayer(snow_mask, vis_params, "Snow Mask")
Map.addLayer(rad.first(), vis_params, "Incoming Rad")

Hour: 10
Count:  2


In [36]:
# -*- coding: utf-8 -*-
"""
Created on Wed Jul 22 17:20:38 2020

@author: kgn
"""
import sys
sys.path.append("C:/Users/andre/OneDrive/Dokumente/Masterarbeit/GEE/Download_Leonie/")
print(sys.path)
import ee
import numpy as np
from osgeo import gdal
from osgeo import osr
    


# Initialize the Earth Engine library.
ee.Initialize()

# fill all NA-values with a fill values of -999999 
def na_to_num(image):
    image_wo_na = image.unmask(-999999)
    return image_wo_na

# mosaic all images on the same date
def helper_function(d, imcol):
    d = ee.Date(d)
    im = imcol.filterDate(d, d.advance(1, "day")).mosaic()
    return im.set("system:time_start", d.millis(), "system:id", d.format("YYYY-MM-dd"))

def mosaic(imcol):
    imlist = imcol.toList(imcol.size()) 
    unique_dates = imlist.map(lambda im: ee.Image(im).date().format("YYYY-MM-dd")).distinct()
    mosaic_imlist = unique_dates.map(lambda d: helper_function(d, imcol))
    return ee.ImageCollection(mosaic_imlist)

def ymdList(imgcol):
    def iter_func(image, newlist):
        date = ee.Number.parse(image.date().format("yyyyMMdd"));
        newlist = ee.List(newlist);
        return ee.List(newlist.add(date).sort())
    ymd = imgcol.iterate(iter_func, ee.List([]))
    return list(ee.List(ymd).reduce(ee.Reducer.frequencyHistogram()).getInfo().keys())


# convert the gee objects into numpys and then into Geotiffs
def gee_to_numpy(img, bounds, poly, indexname):
    path_orig = 'newfiles_'+indexname+'/'
    # get the lat lon and add the ndvi
    latlon = ee.Image.pixelLonLat().addBands(img)

    # apply reducer to list
    latlon = latlon.reduceRegion(
      reducer=ee.Reducer.toList(),
      geometry=bounds,
      maxPixels=1e8,
      scale=500);
    
    # get data into three different arrays 
    data = np.array((ee.Array(latlon.get(indexname)).getInfo()))
    lats = np.array((ee.Array(latlon.get("latitude")).getInfo()))
    lons = np.array((ee.Array(latlon.get("longitude")).getInfo()))
    
   
    # get the unique coordinates
    uniqueLats = np.unique(lats)
    uniqueLons = np.unique(lons)
     
    # get number of columns and rows from coordinates
    ncols = len(uniqueLons)    
    nrows = len(uniqueLats)
     
    # determine pixelsizes
    ys = uniqueLats[1] - uniqueLats[0] 
    xs = uniqueLons[1] - uniqueLons[0]
     
    # create an array with dimensions of image
    arr= np.zeros([nrows, ncols], np.float32) 
  
   # fill the array with values
    counter =0
    for r in range(0,len(arr),1):
        for p in range(0,len(arr[0]),1):
            if lats[counter] == uniqueLats[r] and lons[counter] == uniqueLons[p] and counter < len(lats)-1:
                counter+=1
                arr[len(uniqueLats)-1-r,p] = data[counter] # we start from lower left corner
                    
    # set -999999 to Na
    arr[arr == -999999] = 'nan' 
    
 #   if sat == 'l' or sat == 'm': # create Geotiffs from the landsat and MODIS images at the date 0 
        #SetGeoTransform((originX, pixelWidth, 0, originY, 0, pixelHeight))
    transform = (np.min(uniqueLons),xs,0,np.max(uniqueLats),0,-ys)
     
    # set the coordinate system
    target = osr.SpatialReference()
    target.ImportFromEPSG(4326)
     
    # set driver
    driver = gdal.GetDriverByName('GTiff')
    
    timedate = ee.Date(img.get('system:time_start')).format('yyyyMMdd').getInfo()
    outputDataset = driver.Create(path_orig+timedate+poly+".tif", ncols,nrows, 1,gdal.GDT_Float32)
     
    # add some metadata
    outputDataset.SetMetadata( {'time': str(timedate)} )
     
    outputDataset.SetGeoTransform(transform)
    outputDataset.SetProjection(target.ExportToWkt())
    outputDataset.GetRasterBand(1).WriteArray(arr)
    outputDataset.GetRasterBand(1).SetNoDataValue(-999999)
    outputDataset = None
    return(outputDataset, arr)
   # else:
   #     return(arr)

['C:\\Users\\andre\\OneDrive\\Dokumente\\Masterarbeit\\GEE\\Download_Leonie', 'C:\\Users\\andre\\anaconda3\\envs\\starfm\\python37.zip', 'C:\\Users\\andre\\anaconda3\\envs\\starfm\\DLLs', 'C:\\Users\\andre\\anaconda3\\envs\\starfm\\lib', 'C:\\Users\\andre\\anaconda3\\envs\\starfm', '', 'C:\\Users\\andre\\anaconda3\\envs\\starfm\\lib\\site-packages', 'C:\\Users\\andre\\anaconda3\\envs\\starfm\\lib\\site-packages\\win32', 'C:\\Users\\andre\\anaconda3\\envs\\starfm\\lib\\site-packages\\win32\\lib', 'C:\\Users\\andre\\anaconda3\\envs\\starfm\\lib\\site-packages\\Pythonwin', 'C:\\Users\\andre\\anaconda3\\envs\\starfm\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\andre\\.ipython', 'C:/Users/andre/OneDrive/Dokumente/Masterarbeit/GEE/Download_Leonie/', 'C:/Users/andre/OneDrive/Dokumente/Masterarbeit/GEE/Download_Leonie/', 'C:/Users/andre/OneDrive/Dokumente/Masterarbeit/GEE/Download_Leonie/', 'C:/Users/andre/OneDrive/Dokumente/Masterarbeit/GEE/Download_Leonie/', 'C:/Users/andre/OneDriv

In [37]:
# -*- coding: utf-8 -*-
"""
Created on Thu Sep 10 09:19:52 2020

@author: kgn
"""

import ee
from geetools import cloud_mask
import pandas as pd
#import math


# Initialize the Earth Engine library.
ee.Initialize()

def download_imagery(collection, polygon, polygonname, number_of_processes):
   # modis = modis_col
    bounds = polygon
    #bounds = ee.FeatureCollection("users/leonieanina93/HessenBuff_Sued")
    
    # set the NAs to the numeric fillvalue of -999999
    #modis_wo_na = (modis_final.map(na_to_num))
    col_wo_na = (collection.map(na_to_num))
      
    # create a list from the collection  
    #modis_list = modis_wo_na.toList(modis_wo_na.size())
    col_list = col_wo_na.toList(col_wo_na.size())
    
    # get date from image and create a datelist
    #modis_dates = ymdList(modis_wo_na)
    col_dates = ymdList(col_wo_na)
          
    #create dataframe with dates from images from Landsat and Modis that are used for STARM-fusion   
    #modis_df = pd.DataFrame(
     #      {'modis': modis_dates
      #     })
       
    sat_df = pd.DataFrame(
           {#'modis': landsat_dates,
            'RAD': col_dates
           })
    
  #  sat_df = pd.merge(modis_df, landsat_df, how='left')
    poly = str(polygonname)  #create a polygon label

    sat_df.to_csv(r'.\datetables_RAD\polygon_' + poly + '.csv')
       
    
  #  x = -1
    for j in range(len(sat_df)):
        #print(j)
       # mod_num = int(sat_df.loc[j, "index"])
        #print(mod_num)
       # path_orig = 'original_files/'
        
        #if math.isnan(float(sat_df.loc[j, "landsat"])) == False:
          #  if math.isnan(float(sat_df.loc[j, "landsat"])) == False:
             #  x = sat_df.iloc[j]['landsat']
              #  x = j
          #      x += 1
                #print(x)
                img_l = ee.Image(col_list.get(j)).clip(bounds)
               # print('Print Landsat tif')
                col_tif, arr_l = gee_to_numpy(img_l, bounds, poly, 'RAD')
                
      #  else:
      #          img_m = ee.Image(modis_list.get(j)).clip(bounds)
       #         modis_tif, arr_m = gee_to_numpy(img_m, 'm', bounds, poly, 'NDVI')

In [38]:
download_imagery(rad, geometry, 'aqua', 1)